# Part A : Working with RDDs and DataFrames

## 1. Working with RDD

### 1.1 Data Preparation and Loading

In [1]:
# Import SparkConf class into program
from pyspark import SparkConf

# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[*]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "Introduction to Apache Spark"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

In [2]:
# Import SparkContext and SparkSession classes
from pyspark import SparkContext # Spark
from pyspark.sql import SparkSession # Spark SQL

# # Method 1: Using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

# Method 2: Getting or instantiating a SparkContext
sc = SparkContext.getOrCreate(spark_conf)
sc.setLogLevel('ERROR')

In [3]:
### reading all three files at once
crash_data = sc.textFile('data/2015_DATA_SA_Crash.csv,data/2016_DATA_SA_Crash.csv,data/2017_DATA_SA_Crash.csv', 1 )

In [4]:
# crash_data_1 = sc.parallelize(crash_data,1)

In [5]:
units_data = sc.textFile('data/2015_DATA_SA_Units.csv,data/2016_DATA_SA_Units.csv,data/2017_DATA_SA_Units.csv',1 )

In [6]:
file1 = sc.textFile('data/2015_DATA_SA_Crash.csv',10)

In [7]:
file1.getNumPartitions()

10

In [8]:
## check number of partitions
crash_data.getNumPartitions()

3

In [9]:
units_data.getNumPartitions()

3

In [10]:
crash_data.first()

'"REPORT_ID","Stats Area","Suburb","Postcode","LGA Name","Total Units","Total Cas","Total Fats","Total SI","Total MI","Year","Month","Day","Time","Area Speed","Position Type","Horizontal Align","Vertical Align","Other Feat","Road Surface","Moisture Cond","Weather Cond","DayNight","Crash Type","Unit Resp","Entity Code","CSEF Severity","Traffic Ctrls","DUI Involved","Drugs Involved","ACCLOC_X","ACCLOC_Y","UNIQUE_LOC"'

In [11]:
header = crash_data.first()

In [12]:
header

'"REPORT_ID","Stats Area","Suburb","Postcode","LGA Name","Total Units","Total Cas","Total Fats","Total SI","Total MI","Year","Month","Day","Time","Area Speed","Position Type","Horizontal Align","Vertical Align","Other Feat","Road Surface","Moisture Cond","Weather Cond","DayNight","Crash Type","Unit Resp","Entity Code","CSEF Severity","Traffic Ctrls","DUI Involved","Drugs Involved","ACCLOC_X","ACCLOC_Y","UNIQUE_LOC"'

In [13]:
# remove header
crash_data.filter(lambda line : line != header).take(10)

['"2015-1-21/08/2019","2 Metropolitan","ELIZABETH VALE","5112","CITY OF PLAYFORD.",2,1,0,0,1,2015,"January","Wednesday","01:00 pm","060","T-Junction","Straight road","Level","Not Applicable","Sealed","Dry","Not Raining","Daylight","Side Swipe","01","Driver Rider","2: MI","No Control","","",1335254.54,1690056.88,"13352551690057"',
 '"2015-2-21/08/2019","2 Metropolitan","SALISBURY","5108","CITY OF SALISBURY",2,1,0,0,1,2015,"February","Tuesday","03:38 pm","060","Cross Road","Straight road","Level","Not Applicable","Sealed","Dry","Not Raining","Daylight","Rear End","01","Driver Rider","2: MI","Traffic Signals","","",1333389.6,1688248.34,"13333901688248"',
 '"2015-3-21/08/2019","2 Metropolitan","ST MARYS","5042","CC MITCHAM.                   ",2,1,0,0,1,2015,"March","Tuesday","01:15 pm","070","Cross Road","Straight road","Level","Not Applicable","Sealed","Dry","Not Raining","Daylight","Rear End","01","Driver Rider","2: MI","Traffic Signals","","",1326004.51,1661277.67,"13260051661278"',
 '

In [14]:
# number of records
crash_data.filter(lambda line : line != header).count()

45443

In [15]:
# show top 10 records
units_data.filter(lambda line : line != header).take(10)

['"REPORT_ID","Unit No","No Of Cas","Veh Reg State","Unit Type","Veh Year","Direction Of Travel","Sex","Age","Lic State","Licence Class","Licence Type","Towing","Unit Movement","Number Occupants","Postcode","Rollover","Fire"',
 '"2015-1-21/08/2019","01",0,"SA","RIGID TRUCK LGE GE 4.5T","1999","North East","Male","052","SA","HRR ","Full","Not Towing","Swerving","001","5109",,',
 '"2015-1-21/08/2019","02",1,"SA","Motor Cars - Sedan","2009","North East","Female","057","SA","C ","Full","Not Towing","Straight Ahead","002","5125",,',
 '"2015-2-21/08/2019","01",0,"SA","Motor Cars - Sedan","2009","South East","Male","020","SA","MR","Provisional 1 ","Not Towing","Straight Ahead","001","5110",,',
 '"2015-2-21/08/2019","02",1,"SA","Motor Cars - Sedan","1994","South East","Female","021","SA","C ","Full","Not Towing","Stopped on Carriageway","001","5096",,',
 '"2015-3-21/08/2019","01",0,"SA","Motor Cars - Sedan","2008","North East","Male","023","SA","C ","Full","Not Towing","Straight Ahead","001","

In [16]:
units_data.filter(lambda line : line != header).count()

97111

#### 1.2

In [17]:
units_data.getNumPartitions()

3

By default, it is set to the total number of cores on all the executor nodes

In [18]:
header = units_data.first()

In [19]:
rdd1 = units_data.filter(lambda x: x != header)


In [20]:
# rdd1[0].take(1)

In [21]:
# Implement function with logic to be applied to the RDDs
def parseRecord(line):
    # Split line separated by comma
    array_line = line.split(',')

    rest_array = array_line
    del rest_array[8]
    # return tuple 
    return (array_line[8], rest_array[0:])

In [22]:
rdd2 = rdd1.map(parseRecord)

In [23]:
rdd2.take(5)

[('"SA"',
  ['"2015-1-21/08/2019"',
   '"01"',
   '0',
   '"SA"',
   '"RIGID TRUCK LGE GE 4.5T"',
   '"1999"',
   '"North East"',
   '"Male"',
   '"SA"',
   '"HRR "',
   '"Full"',
   '"Not Towing"',
   '"Swerving"',
   '"001"',
   '"5109"',
   '',
   '']),
 ('"SA"',
  ['"2015-1-21/08/2019"',
   '"02"',
   '1',
   '"SA"',
   '"Motor Cars - Sedan"',
   '"2009"',
   '"North East"',
   '"Female"',
   '"SA"',
   '"C "',
   '"Full"',
   '"Not Towing"',
   '"Straight Ahead"',
   '"002"',
   '"5125"',
   '',
   '']),
 ('"SA"',
  ['"2015-2-21/08/2019"',
   '"01"',
   '0',
   '"SA"',
   '"Motor Cars - Sedan"',
   '"2009"',
   '"South East"',
   '"Male"',
   '"SA"',
   '"MR"',
   '"Provisional 1 "',
   '"Not Towing"',
   '"Straight Ahead"',
   '"001"',
   '"5110"',
   '',
   '']),
 ('"SA"',
  ['"2015-2-21/08/2019"',
   '"02"',
   '1',
   '"SA"',
   '"Motor Cars - Sedan"',
   '"1994"',
   '"South East"',
   '"Female"',
   '"SA"',
   '"C "',
   '"Full"',
   '"Not Towing"',
   '"Stopped on Carriagew

In [24]:
rdd_grouped = rdd2.groupByKey(2)

In [25]:
rdd2.getNumPartitions()

3

In [26]:
def count_in_a_partition(idx, iterator):
  count = 0
  for _ in iterator:
    count += 1
  return idx, count

In [27]:
rdd2.mapPartitionsWithIndex(count_in_a_partition).collect()

[0, 33084, 1, 35861, 2, 28163]

### 1.3

In [28]:
rdd3 = units_data.map(lambda line: line.split(',')).map(lambda x: (x[7], x[8]))

In [29]:
rdd3.take(3)

[('"Sex"', '"Age"'), ('"Male"', '"052"'), ('"Female"', '"057"')]

In [30]:
header = rdd3.first()
header

('"Sex"', '"Age"')

In [31]:
rdd3_filtered = rdd3.filter(lambda x: x != header)

In [32]:
rdd3_filtered.take(3)

[('"Male"', '"052"'), ('"Female"', '"057"'), ('"Male"', '"020"')]

In [33]:
rdd3_male = rdd3.filter(lambda x: x[0] == '"Male"')

In [34]:
rdd3_male.take(3)

[('"Male"', '"052"'), ('"Male"', '"020"'), ('"Male"', '"023"')]

In [35]:
rdd3_female = rdd3.filter(lambda x: x[0] == '"Female"')

In [36]:
rdd3_female.take(1)

[('"Female"', '"057"')]

In [37]:
rdd3_male = rdd3_male.map(lambda x: x[1].lstrip('"').rstrip('"'))

In [38]:
rdd3_female = rdd3_female.map(lambda x: x[1].lstrip('"').rstrip('"'))

In [39]:
age = 0;
count = 0
for item in rdd3_male.collect(): 
    try: 
        age += int(item)
        count += 1
    except : 
        None
#         print(item)

In [40]:
avg_age_male = age/count
avg_age_male

40.81305841924399

In [41]:
age = 0;
count = 0
for item in rdd3_female.collect(): 
    try: 
        age += int(item)
        count += 1
    except : 
        None
#         print(item)

In [42]:
avg_age_female = age/count
avg_age_female

40.185663534645904

In [43]:
units_data.take(3)

['"REPORT_ID","Unit No","No Of Cas","Veh Reg State","Unit Type","Veh Year","Direction Of Travel","Sex","Age","Lic State","Licence Class","Licence Type","Towing","Unit Movement","Number Occupants","Postcode","Rollover","Fire"',
 '"2015-1-21/08/2019","01",0,"SA","RIGID TRUCK LGE GE 4.5T","1999","North East","Male","052","SA","HRR ","Full","Not Towing","Swerving","001","5109",,',
 '"2015-1-21/08/2019","02",1,"SA","Motor Cars - Sedan","2009","North East","Female","057","SA","C ","Full","Not Towing","Straight Ahead","002","5125",,']

In [44]:
rdd4.take(2)

NameError: name 'rdd4' is not defined

In [45]:
header = units_data.first()

In [46]:
rdd4 = units_data.map(lambda line: line.split(',')).filter(lambda line : line != header).map(lambda x: (x[0],x[1:]))

In [47]:
# crash_data.filter(lambda line : line != header).take(10)

In [48]:
rdd4.take(2)

[('"REPORT_ID"',
  ['"Unit No"',
   '"No Of Cas"',
   '"Veh Reg State"',
   '"Unit Type"',
   '"Veh Year"',
   '"Direction Of Travel"',
   '"Sex"',
   '"Age"',
   '"Lic State"',
   '"Licence Class"',
   '"Licence Type"',
   '"Towing"',
   '"Unit Movement"',
   '"Number Occupants"',
   '"Postcode"',
   '"Rollover"',
   '"Fire"']),
 ('"2015-1-21/08/2019"',
  ['"01"',
   '0',
   '"SA"',
   '"RIGID TRUCK LGE GE 4.5T"',
   '"1999"',
   '"North East"',
   '"Male"',
   '"052"',
   '"SA"',
   '"HRR "',
   '"Full"',
   '"Not Towing"',
   '"Swerving"',
   '"001"',
   '"5109"',
   '',
   ''])]

### 2

#### 2.1

Loading Data

In [49]:
df = spark.read.format('csv')\
            .option('header',True).option('escape','"')\
            .load('data/2015_DATA_SA_Crash.csv')
df.count()

15431

Displaying Schema and Count

In [50]:
df_crash = spark.read.format('csv')\
            .option('header',True).option('escape','"')\
            .load('data/*_DATA_SA_Crash.csv')
print(df_crash.count())
df_crash.printSchema()

72006
root
 |-- REPORT_ID: string (nullable = true)
 |-- Stats Area: string (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- LGA Name: string (nullable = true)
 |-- Total Units: string (nullable = true)
 |-- Total Cas: string (nullable = true)
 |-- Total Fats: string (nullable = true)
 |-- Total SI: string (nullable = true)
 |-- Total MI: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Area Speed: string (nullable = true)
 |-- Position Type: string (nullable = true)
 |-- Horizontal Align: string (nullable = true)
 |-- Vertical Align: string (nullable = true)
 |-- Other Feat: string (nullable = true)
 |-- Road Surface: string (nullable = true)
 |-- Moisture Cond: string (nullable = true)
 |-- Weather Cond: string (nullable = true)
 |-- DayNight: string (nullable = true)
 |-- Crash Type: string (nullable = true)


In [51]:
df_units = spark.read.format('csv')\
            .option('header',True).option('escape','"')\
            .load('data/*_DATA_SA_Units.csv')
print(df_units.count())
df_units.printSchema()

153854
root
 |-- REPORT_ID: string (nullable = true)
 |-- Unit No: string (nullable = true)
 |-- No Of Cas: string (nullable = true)
 |-- Veh Reg State: string (nullable = true)
 |-- Unit Type: string (nullable = true)
 |-- Veh Year: string (nullable = true)
 |-- Direction Of Travel: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Lic State: string (nullable = true)
 |-- Licence Class: string (nullable = true)
 |-- Licence Type: string (nullable = true)
 |-- Towing: string (nullable = true)
 |-- Unit Movement: string (nullable = true)
 |-- Number Occupants: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Rollover: string (nullable = true)
 |-- Fire: string (nullable = true)



In [52]:
# import pyspark.sql.functions as F


#### 2.2

In [53]:
df_crash.na.drop()

DataFrame[REPORT_ID: string, Stats Area: string, Suburb: string, Postcode: string, LGA Name: string, Total Units: string, Total Cas: string, Total Fats: string, Total SI: string, Total MI: string, Year: string, Month: string, Day: string, Time: string, Area Speed: string, Position Type: string, Horizontal Align: string, Vertical Align: string, Other Feat: string, Road Surface: string, Moisture Cond: string, Weather Cond: string, DayNight: string, Crash Type: string, Unit Resp: string, Entity Code: string, CSEF Severity: string, Traffic Ctrls: string, DUI Involved: string, Drugs Involved: string, ACCLOC_X: string, ACCLOC_Y: string, UNIQUE_LOC: string]

In [54]:
df_crash.select('Suburb').show()

+---------------+
|         Suburb|
+---------------+
|       ADELAIDE|
|        POORAKA|
|   GREEN FIELDS|
|  MITCHELL PARK|
| EVANSTON SOUTH|
|   MOUNT BARKER|
|      DRY CREEK|
|       PAYNEHAM|
|       ADELAIDE|
|      THEBARTON|
|       ADELAIDE|
| MILE END SOUTH|
| CHAIN OF PONDS|
|    CAPE JERVIS|
| WOODVILLE WEST|
|        HACKNEY|
|WESTBOURNE PARK|
| WILLUNGA SOUTH|
|         JOSLIN|
|       ADELAIDE|
+---------------+
only showing top 20 rows



Find all the crash events in Adelaide where the total number of casualties in the event
is more than 3

In [55]:
filtered = df_crash.filter(df_crash.Suburb == 'ADELAIDE').filter(df_crash['Total Cas'] > 3)

In [56]:
filtered.show()

+--------------------+----------+--------+--------+----------------+-----------+---------+----------+--------+--------+----+--------+--------+--------+----------+-------------+----------------+--------------+--------------------+------------+-------------+------------+--------+--------------+---------+------------+-------------+---------------+------------+--------------+----------+----------+--------------+
|           REPORT_ID|Stats Area|  Suburb|Postcode|        LGA Name|Total Units|Total Cas|Total Fats|Total SI|Total MI|Year|   Month|     Day|    Time|Area Speed|Position Type|Horizontal Align|Vertical Align|          Other Feat|Road Surface|Moisture Cond|Weather Cond|DayNight|    Crash Type|Unit Resp| Entity Code|CSEF Severity|  Traffic Ctrls|DUI Involved|Drugs Involved|  ACCLOC_X|  ACCLOC_Y|    UNIQUE_LOC|
+--------------------+----------+--------+--------+----------------+-----------+---------+----------+--------+--------+----+--------+--------+--------+----------+-------------+

Display 10 crash events with highest ​ casualties​

In [57]:
from pyspark.sql.functions import col
df_crash.sort(col("Total Cas").desc()).show(10)

+--------------------+--------------+---------------+--------+--------------------+-----------+---------+----------+--------+--------+----+--------+---------+--------+----------+-------------+--------------------+--------------+--------------+------------+-------------+------------+--------+-----------+---------+------------+-------------+---------------+------------+--------------+----------+----------+--------------+
|           REPORT_ID|    Stats Area|         Suburb|Postcode|            LGA Name|Total Units|Total Cas|Total Fats|Total SI|Total MI|Year|   Month|      Day|    Time|Area Speed|Position Type|    Horizontal Align|Vertical Align|    Other Feat|Road Surface|Moisture Cond|Weather Cond|DayNight| Crash Type|Unit Resp| Entity Code|CSEF Severity|  Traffic Ctrls|DUI Involved|Drugs Involved|  ACCLOC_X|  ACCLOC_Y|    UNIQUE_LOC|
+--------------------+--------------+---------------+--------+--------------------+-----------+---------+----------+--------+--------+----+--------+------

In [58]:
from pyspark.sql.types import IntegerType
df_crash_num = df_crash.withColumn("Total Fats", df_crash["Total Fats"].cast(IntegerType()))
df_crash_num = df_crash_num.withColumn("Total Cas", df_crash["Total Cas"].cast(IntegerType()))

In [59]:
df_crash_type = df_crash_num.groupBy('Crash Type')

In [60]:
# df_crash_type = df_crash_type.withColumn("Total Fats", df_crash_type["Total Fats"].cast(IntegerType()))

Find the total number of ​ fatalities ​ for each crash type

In [61]:
df_crash_type.sum('Total Fats').show()

+--------------------+---------------+
|          Crash Type|sum(Total Fats)|
+--------------------+---------------+
|           Roll Over|             57|
|  Hit Object on Road|              2|
|      Hit Pedestrian|             70|
|    Hit Fixed Object|            152|
|               Other|              2|
|          Side Swipe|             20|
|             Head On|             86|
|  Hit Parked Vehicle|              9|
|          Right Turn|             18|
|            Rear End|             16|
|          Hit Animal|              4|
|Left Road - Out o...|              1|
|         Right Angle|             45|
+--------------------+---------------+



In [62]:
new_df = df_crash_num.join(df_units, on=['REPORT_ID'], how='right_outer')

In [63]:
# new_df.show()

Find the total number of casualties for each suburb when the vehicle was driven by an
unlicensed driver

In [64]:
df_crash_suburb = new_df.where(col('Licence Type').isNull()).groupBy('Suburb').sum('Total Cas').show()

+-------------------+--------------+
|             Suburb|sum(Total Cas)|
+-------------------+--------------+
|      FLINDERS PARK|            31|
|     TEA TREE GULLY|            14|
|       POOGINAGORIC|             2|
|   HALLELUJAH HILLS|             0|
|            HACKHAM|             9|
|         SALT CREEK|             6|
|       BASKET RANGE|             4|
|            CUMMINS|             3|
|       POINT PEARCE|             2|
|           WISANGER|             6|
|   MEDINDIE GARDENS|             0|
|      MOUNT CHARLES|             1|
|             HAWKER|             0|
|          STANSBURY|             1|
|         MOUNT BURR|             2|
|MURRAY BRIDGE SOUTH|             0|
|        EAST MOONTA|             1|
|      GILLES PLAINS|            14|
|           MUNDOORA|             2|
|       BULLOO CREEK|             3|
+-------------------+--------------+
only showing top 20 rows



In [65]:
# df_crash_suburb.select('Licence Type').collect()

### 2.3

Find the total number of crash events for each severity level. Which severity level is the
most common

In [66]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window
df_crash_num.groupBy('CSEF Severity').count().withColumn('percentage', f.round(f.col('count') / f.sum('count')\
  .over(Window.partitionBy()),3)).show()

+-------------+-----+----------+
|CSEF Severity|count|percentage|
+-------------+-----+----------+
|     4: Fatal|  451|     0.006|
|        2: MI|21881|     0.304|
|       1: PDO|46696|     0.649|
|        3: SI| 2978|     0.041|
+-------------+-----+----------+



PDO- property damage only has most number of recorded cases therefore it is the most common.

Compute the total number of crash events for each severity level and the percentage
for the four different scenarios.

In [67]:
df_crash_num_alc = df_crash_num.where(col('DUI Involved').isNotNull())

In [68]:
df_crash_num_alc.groupBy('CSEF Severity').count().withColumn('percentage', f.round(f.col('count') / f.sum('count')\
  .over(Window.partitionBy()),3)).show()

+-------------+-----+----------+
|CSEF Severity|count|percentage|
+-------------+-----+----------+
|     4: Fatal|   79|     0.035|
|        2: MI|  737|     0.328|
|       1: PDO| 1173|     0.522|
|        3: SI|  259|     0.115|
+-------------+-----+----------+



In [69]:
df_crash_num_drugs = df_crash_num.where(col('Drugs Involved').isNotNull())
df_crash_num_drugs.groupBy('CSEF Severity').count().withColumn('percentage', f.round(f.col('count') / f.sum('count')\
  .over(Window.partitionBy()),3)).show()

+-------------+-----+----------+
|CSEF Severity|count|percentage|
+-------------+-----+----------+
|     4: Fatal|   82|     0.065|
|        2: MI|  749|     0.597|
|       1: PDO|  176|      0.14|
|        3: SI|  247|     0.197|
+-------------+-----+----------+



In [70]:
df_crash_drugs_alc = df_crash_num_drugs.where(col('DUI Involved').isNotNull())
df_crash_drugs_alc.groupBy('CSEF Severity').count().withColumn('percentage', f.round(f.col('count') / f.sum('count')\
  .over(Window.partitionBy()),3)).show()

+-------------+-----+----------+
|CSEF Severity|count|percentage|
+-------------+-----+----------+
|     4: Fatal|   27|     0.154|
|        2: MI|   89|     0.509|
|       1: PDO|   24|     0.137|
|        3: SI|   35|       0.2|
+-------------+-----+----------+



In [71]:
df_crash_no_drug_alc = df_crash_num.where(col('DUI Involved').isNull()).where(col('Drugs Involved').isNull())
df_crash_no_drug_alc.groupBy('CSEF Severity').count().withColumn('percentage', f.round(f.col('count') / f.sum('count')\
  .over(Window.partitionBy()),3)).show()

+-------------+-----+----------+
|CSEF Severity|count|percentage|
+-------------+-----+----------+
|     4: Fatal|  317|     0.005|
|        2: MI|20484|     0.298|
|       1: PDO|45371|     0.661|
|        3: SI| 2507|     0.037|
+-------------+-----+----------+



### 2.4

Find the total number of casualties for each suburb when the vehicle was driven by an
unlicensed driver.

In [72]:
%%time 
df_unit_adelaide = new_df.where(col('Suburb') == 'ADELAIDE').select('REPORT_ID','Unit No', 'Time', 'Total Cas', 'Age', 'Licence Type').show()

+------------------+-------+--------+---------+----+------------+
|         REPORT_ID|Unit No|    Time|Total Cas| Age|Licence Type|
+------------------+-------+--------+---------+----+------------+
| 2016-1-15/08/2019|     01|01:45 pm|        1| 056|        Full|
| 2016-1-15/08/2019|     02|01:45 pm|        1| 072|        null|
| 2016-9-15/08/2019|     01|03:40 pm|        1| 056|        null|
| 2016-9-15/08/2019|     02|03:40 pm|        1| 027|        null|
|2016-11-15/08/2019|     01|05:00 pm|        0| 032|        Full|
|2016-11-15/08/2019|     02|05:00 pm|        0| XXX|     Unknown|
|2016-20-15/08/2019|     01|05:40 pm|        0| 022|     Unknown|
|2016-20-15/08/2019|     02|05:40 pm|        0| 020|     Unknown|
|2016-26-15/08/2019|     01|11:26 pm|        0| XXX|     Unknown|
|2016-26-15/08/2019|     02|11:26 pm|        0| 042|        Full|
|2016-26-15/08/2019|     03|11:26 pm|        0|null|        null|
|2016-27-15/08/2019|     01|11:30 pm|        0| 026|     Unknown|
|2016-27-1

In [73]:
new_df.createOrReplaceTempView("sql_events")


In [74]:
sql_count = spark.sql('''
  SELECT `REPORT_ID`, `Unit no`, `Time`, `Total Cas`, `Age`, `Licence Type`
  FROM sql_events
''')

In [75]:
%%time
sql_count.show()

+-----------------+-------+--------+---------+----+-------------+
|        REPORT_ID|Unit no|    Time|Total Cas| Age| Licence Type|
+-----------------+-------+--------+---------+----+-------------+
|2016-1-15/08/2019|     01|01:45 pm|        1| 056|         Full|
|2016-1-15/08/2019|     02|01:45 pm|        1| 072|         null|
|2016-2-15/08/2019|     01|01:00 pm|        0| 023|         Full|
|2016-2-15/08/2019|     02|01:00 pm|        0| 040|         Full|
|2016-3-15/08/2019|     01|07:10 am|        0| XXX|Provisional 2|
|2016-3-15/08/2019|     02|07:10 am|        0| 023|         Full|
|2016-4-15/08/2019|     01|10:30 am|        0| 033|         Full|
|2016-4-15/08/2019|     02|10:30 am|        0| XXX|      Unknown|
|2016-5-15/08/2019|     01|12:56 pm|        3| 042|         Full|
|2016-5-15/08/2019|     02|12:56 pm|        3| 059|         Full|
|2016-6-15/08/2019|     01|02:30 pm|        0| 035|         Full|
|2016-6-15/08/2019|     02|02:30 pm|        0| 042|         Full|
|2016-7-15

In [76]:
%%time 
df_crash_suburb = new_df.where(col('Licence Type').isNull()).groupBy('Suburb').sum('Total Cas').show()

+-------------------+--------------+
|             Suburb|sum(Total Cas)|
+-------------------+--------------+
|      FLINDERS PARK|            31|
|     TEA TREE GULLY|            14|
|       POOGINAGORIC|             2|
|   HALLELUJAH HILLS|             0|
|            HACKHAM|             9|
|         SALT CREEK|             6|
|       BASKET RANGE|             4|
|            CUMMINS|             3|
|       POINT PEARCE|             2|
|           WISANGER|             6|
|   MEDINDIE GARDENS|             0|
|      MOUNT CHARLES|             1|
|             HAWKER|             0|
|          STANSBURY|             1|
|         MOUNT BURR|             2|
|MURRAY BRIDGE SOUTH|             0|
|        EAST MOONTA|             1|
|      GILLES PLAINS|            14|
|           MUNDOORA|             2|
|       BULLOO CREEK|             3|
+-------------------+--------------+
only showing top 20 rows

CPU times: user 6.56 ms, sys: 107 µs, total: 6.67 ms
Wall time: 2.55 s


In [77]:
new_df.createOrReplaceTempView("sql_events")


In [78]:

sql_count = spark.sql('''
  SELECT Suburb, sum(`Total Cas`)
  FROM sql_events where `Licence Type` IS NULL
  GROUP BY Suburb
''')

In [79]:
%%time
sql_count.show()

+-------------------+--------------+
|             Suburb|sum(Total Cas)|
+-------------------+--------------+
|      FLINDERS PARK|            31|
|     TEA TREE GULLY|            14|
|       POOGINAGORIC|             2|
|   HALLELUJAH HILLS|             0|
|            HACKHAM|             9|
|         SALT CREEK|             6|
|       BASKET RANGE|             4|
|            CUMMINS|             3|
|       POINT PEARCE|             2|
|           WISANGER|             6|
|   MEDINDIE GARDENS|             0|
|      MOUNT CHARLES|             1|
|             HAWKER|             0|
|          STANSBURY|             1|
|         MOUNT BURR|             2|
|MURRAY BRIDGE SOUTH|             0|
|        EAST MOONTA|             1|
|      GILLES PLAINS|            14|
|           MUNDOORA|             2|
|       BULLOO CREEK|             3|
+-------------------+--------------+
only showing top 20 rows

CPU times: user 964 µs, sys: 177 µs, total: 1.14 ms
Wall time: 1.92 s


In [80]:
total_data = units_data.union(crash_data)